In [1]:
import pandas as pd
import numpy as np
import math

# Listings' files

First let's define some useful lists and dictionaries

In [2]:
string_attributes = ['listing_url', 'name', 'summary',"space", "description", "notes",\
                        "transit","access","interaction","picture_url","neighbourhood_overview", "neighbourhood",\
                        "host_url", "host_name", "host_about", "host_thumbnail_url", "host_picture_url", \
                        "host_verifications", "property_type", "room_type", "bed_type",\
                        "amenities", "house_rules", "cancellation_policy", "city", "country"]
bit_attributes = ['is_business_travel_ready', 'require_guest_profile_picture', 'require_guest_phone_verification']

date_attributes = ["review_date", "host_since"]

int_attributes = ['id', 'host_id', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included',\
                    'minimum_nights', 'maximum_nights']

In [3]:
data_types = {}
for a in string_attributes:
    data_types[a] = str

For now (as testing purpose) let's use the bracelona file only

In [4]:
filename = "../Dataset/barcelona_listings.csv"


file = open(filename, newline='')
df = pd.read_csv(filename, dtype=data_types)

In [22]:
tables_to_attributes = \
    {"Listing": {"listing_id": "id", "listing_url": "listing_url", "listing_name": "name", "listing_summary": "summary", "listing_space": "space", "listing_description": "description", "listing_notes": "notes", "listing_transit": "transit", "listing_access": "access", "listing_interaction": "interaction","listing_picture_url": "picture_url", "listing_neighbourhood_overview" : "neighbourhood_overview",\
     "accommodates": "accommodates", "bathrooms": "bathrooms", "bedrooms": "bedrooms", "beds": "beds", "square_feet": "square_feet",\
     "price": "price", "weekly_price": "weekly_price", "monthly_price": "monthly_price", "security_deposit": "security_deposit", "cleaning_fee": "cleaning_fee", "guests_included": "guests_included", "extra_people": "extra_people",\
     "rules": "house_rules", "minimum_nights": "minimum_nights", "maximum_nights": "maximum_nights", "is_business_travel_ready": "is_business_travel_ready", "require_guest_profile_picture": "require_guest_profile_picture", "require_guest_phone_verification": "require_guest_phone_verification",\
     "review_scores_rating": "review_scores_rating", "review_scores_accuracy": "review_scores_accuracy", "review_scores_cleanliness": "review_scores_cleanliness", "review_scores_checkin": "review_scores_checkin", "review_scores_communication": "review_scores_communication", "review_scores_location": "review_scores_location", "review_scores_value": "review_scores_value",\
     "latitude": "latitude", "longitude": "longitude",\
     "host_id": "host_id", "neighbourhood_id": "neighbourhood_id", "property_type_id": "property_type_id", "room_type_id": "room_type_id", "bed_type_id": "bed_type_id", "cancellation_policy_id": "cancellation_policy_id"},\
     "Host": {"host_id" : "host_id", "host_url" : "host_url", "host_name" : "host_name", "host_since" : "host_since", "host_about" : "host_about", "host_response_time" : "host_response_time", "host_response_rate" : "host_response_rate", "host_thumbnail_url" : "host_thumbnail_url", "host_picture_url" : "host_picture_url", "neighbourhood_id": "neighbourhood_id"},\
     "Neighbourhood": {"neighbourhood_id":"neighbourhood_id", "neighbourhood_name":"neighbourhood_name","city_id": "city_id"},\
     "City": {"city_id": "city_id", "city_name": "city", "country_id": "country_id"},\
     "Property_type": {"property_type_id": "property_type_id", "property_type_name": "property_type_name"}, \
     "Room_type" : {"room_type_id": "room_type_id", "room_type_name": "room_type_name"},\
     "Bed_type": {"bed_type_id": "bed_type_id", "bed_type_name": "bed_type_name"},\
     "Cancellation_policy": {"cancellation_policy_id": "cancellation_policy_id", "cancellation_policy_name": "cancellation_policy_name"},\
     "Country": {"country_id": "country_id", "country_code": "country_code", "country_name": "country_name"},\
     "Amenity": {"amenity_id": "amenity_id", "amenity_name": "amenity_name"},\
     "Host_verification": {"host_verification_id": "host_verification_id", "host_verification_description": "host_verification_description"},\
     "Listing_amenity_map": {"listing_id": "listing_id", "amenity_id": "amenity_id"},\
     "Host_verification_map" :{"host_id": "host_id", "host_verification_id": "host_verification_id"}}


df = df.rename(columns = {'neighborhood_overview': 'neighbourhood_overview'})

In [24]:
listings_atttributes = list(tables_to_attributes["Listing"].values())

listings_atttributes.remove("neighbourhood_id")
listings_atttributes.remove("property_type_id")
listings_atttributes.remove("room_type_id")
listings_atttributes.remove("bed_type_id")
listings_atttributes.remove("cancellation_policy_id")

normalized_attr = []
normalized_attr.append("neighbourhood")
normalized_attr.append("property_type")
normalized_attr.append("room_type")
normalized_attr.append("bed_type")
normalized_attr.append("cancellation_policy")

listings_atttributes += normalized_attr
print(listings_atttributes)

listings = df[listings_atttributes]
listings = listings.drop_duplicates()
listings['property_type']


['id', 'listing_url', 'name', 'summary', 'space', 'description', 'notes', 'transit', 'access', 'interaction', 'picture_url', 'neighbourhood_overview', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'square_feet', 'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'house_rules', 'minimum_nights', 'maximum_nights', 'is_business_travel_ready', 'require_guest_profile_picture', 'require_guest_phone_verification', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'latitude', 'longitude', 'host_id', 'neighbourhood', 'property_type', 'room_type', 'bed_type', 'cancellation_policy']


0                Apartment
1                Apartment
2                Apartment
3                Apartment
4                Apartment
5                Apartment
6                Apartment
7                Apartment
8                Apartment
9                Apartment
10               Apartment
11               Apartment
12               Apartment
13       Bed and breakfast
14       Bed and breakfast
15       Bed and breakfast
16       Bed and breakfast
17               Apartment
18                    Loft
19               Apartment
20               Apartment
21               Apartment
22               Apartment
23               Apartment
24               Apartment
25               Apartment
26               Apartment
27               Apartment
28               Apartment
29       Bed and breakfast
               ...        
13065            Apartment
13066            Apartment
13067               Hostel
13068            Apartment
13069               Hostel
13070            Apartment
1

In [10]:
def clean_float_and_int(a):
    a = str(a)
    a = a.replace("$", "")
    a = a.replace(",", "")
    return float(a)